In [1]:
from model.types import MarketDemandSupply
from model.run import load_config, run, post_processing

from model.mechanism.amm_k import amm_k_mechanism
from model.mechanism.treasury import liq_backing_mechanism,treasury_stables_mechanism,floating_supply_mechanism,liq_ohm_mechanism
from model.policy.rbs_price import lower_target_policy,upper_target_policy

params = {"demand_factor": [10],
         "supply_factor": [10],
         "initial_reserves_volatile":[17000000], # setting it as constant since in this simulation we don't use it at all.
         "reward_rate":[0.000198], # flat reward rate; (1 + r) ^ 365 ~ 7.5%
         "max_liq_ratio":[0.14375],  # LiquidityUSD : reservesUSD ratio --> 1:1 = 0.5
         "target_ma":[30], # number of days
         "lower_wall":[0.15],
         "upper_wall":[0.15],
         "lower_cushion":[.075],
         "upper_cushion":[.075],
          "reinstate_window": [30]
         }


initialstate = { # variable values borrowed from liquidity-olympus/simulation.ipynb
            'supply':30000000,
            "liq_stables": 21000000,
            "reserves_stables":200000000,
            "reserves_volatile":17000000,
            "price":9,

            "market_demand_supply": MarketDemandSupply(total_supply = 50,
                                                   total_demand = 50),
                                                   
            # treasury policy
            'target_liq_ratio_reached':False,
            'reserves_in':0,
            'supply':25000000,

            # RBS related
            'ma_target':9,
            # market transaction variables
            'ask_change_ohm':0,
            'bid_change_ohm':0,
            "net_flow": None
            }
# initialize parameters that are decided by other parameters
initialstate['amm_k'] = amm_k_mechanism(initialstate)
initialstate['price_history']=[initialstate['price']]

initialstate['lb_target'] = initialstate['ma_target']
initialstate['price_target']=initialstate['ma_target']

initialstate['liq_ohm'] = liq_ohm_mechanism(initialstate['liq_stables'],initialstate['price'])
initialstate['floating_supply'] = floating_supply_mechanism(initialstate['supply'],initialstate['liq_ohm'])



initialstate['treasury_stables'] = treasury_stables_mechanism(initialstate['liq_stables'],initialstate['reserves_stables'])
initialstate['liq_backing'] = liq_backing_mechanism(initialstate['treasury_stables'],params['initial_reserves_volatile'][0])
initialstate['target_walls'] = [lower_target_policy(initialstate['price_target'], params['lower_wall'][0]),
                                upper_target_policy(initialstate['price_target'], params['upper_wall'][0])]
initialstate['target_cushions'] = [lower_target_policy(initialstate['price_target'], params['lower_cushion'][0]),
                                    upper_target_policy(initialstate['price_target'], params['upper_cushion'][0])]
initialstate['bid_counter'],initialstate['ask_counter'] = [0] * params["reinstate_window"][0],[0]* params["reinstate_window"][0]

nrun = 5
nstep = 10
exp = load_config(nrun, params, initialstate, nstep)
raw = run(exp)
df = post_processing(raw)


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

cadCAD Version: 0.4.28
Execution Mode: local_proc
Simulation Dimensions:
Entire Simulation: (Models, Unique Timesteps, Params, Total Runs, Sub-States) = (1, 10, 11, 5, 24)
     Simulation 0: (Timesteps, Params, Runs, Sub-States) = (10, 11, 5, 24)
Execution Method: local_simulations
Execution Mode: parallelized
Total execution time: 0.47s


In [2]:
raw.columns

Index(['supply', 'liq_stables', 'reserves_stables', 'reserves_volatile',
       'price', 'market_demand_supply', 'target_liq_ratio_reached',
       'reserves_in', 'ma_target', 'ask_change_ohm', 'bid_change_ohm',
       'net_flow', 'amm_k', 'price_history', 'lb_target', 'price_target',
       'liq_ohm', 'floating_supply', 'treasury_stables', 'liq_backing',
       'target_walls', 'target_cushions', 'bid_counter', 'ask_counter',
       'simulation', 'subset', 'run', 'substep', 'timestep'],
      dtype='object')

# Check Summary

1. Changed s_liq_backing to use the state instead of parameters for reserves_volatile.
2. Ordering needs to be changed of substeps
3. Added in re-instate window parameter
4. Initialized bid/ask counter based on re-instate window with 0. Was leading to an error before because the length of the array was only 1 all the way through.


# Checks Questions

1. Why is it price_target vs. ma_target for p_ask_counter? It's ma_target in the code, are you going off of the doc or something else?

In [3]:
left = raw[raw["substep"] == 1][['liq_stables', 'reserves_stables']].sum(axis=1)
right = raw[raw["substep"] == 1]['treasury_stables']

assert (left == right).all()

In [4]:
left = raw[raw["substep"] == 2][['liq_stables', 'reserves_stables', 'reserves_volatile']].sum(axis=1)
right = raw[raw["substep"] == 2]['liq_backing']

assert (left == right).all()

In [5]:
left = raw[raw["substep"] == 3]['price_history'].apply(lambda x: len(x))
right = raw[raw["substep"] == 3]['timestep'] + 1

assert (left == right).all()

In [6]:
sample = raw[raw["substep"] == 4]

# Check length
assert (sample['bid_counter'].apply(lambda x: len(x)) == 30).all()

# Check last value
left = sample['bid_counter'].apply(lambda x: x[-1])
right = sample["price"] > sample['ma_target']

assert (left == right).all()

In [7]:
sample = raw[raw["substep"] == 5]

# Check length
assert (sample['ask_counter'].apply(lambda x: len(x)) == 30).all()

# Check last value
left = sample['ask_counter'].apply(lambda x: x[-1])
right = sample["price"] < sample['ma_target']

assert (left == right).all()

AssertionError: 

In [12]:
sample["price_target"]

5       9.0
17     10.5
29     10.5
41     10.5
53     10.5
65     10.5
77     10.5
89     10.5
101    10.5
113    10.5
126     9.0
138    10.5
150    10.5
162    10.5
174    10.5
186    10.5
198    10.5
210    10.5
222    10.5
234    10.5
247     9.0
259    10.5
271    10.5
283    10.5
295    10.5
307    10.5
319    10.5
331    10.5
343    10.5
355    10.5
368     9.0
380    10.5
392    10.5
404    10.5
416    10.5
428    10.5
440    10.5
452    10.5
464    10.5
476    10.5
489     9.0
501    10.5
513    10.5
525    10.5
537    10.5
549    10.5
561    10.5
573    10.5
585    10.5
597    10.5
Name: price_target, dtype: float64